In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
import torch
import random
import numpy as np
import pandas as pd
import torchmetrics
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader

In [3]:
TRAIN_IMAGES_PATH = 'train.h5'
VAL_IMAGES_PATH = 'val.h5'
TEST_IMAGES_PATH = 'test.h5'
SEED = 0

In [4]:
torch.random.manual_seed(SEED)
random.seed(SEED)

In [15]:
from DLMI_histopathology.dataset import BaselineDataset, PrecomputedDataset,precompute

In [6]:
# Statistics on train set

mean = [0.7439, 0.5892, 0.7210] 
std = [0.1717, 0.2065, 0.1664]

# Create the normalization transform
transform = transforms.Compose([
            transforms.Resize((98, 98)),
            transforms.Normalize(mean=mean, std=std)
        ])



In [7]:
train_dataset = BaselineDataset(TRAIN_IMAGES_PATH, transform, 'train')
val_dataset = BaselineDataset(VAL_IMAGES_PATH, transform, 'train')

In [9]:
from DLMI_histopathology.train import train_model

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Working on {device}.')

Working on cuda.


In [11]:
feature_extractor = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to(device)
feature_extractor.eval()

Using cache found in /raid/home/detectionfeuxdeforet/caillaud_gab/.cache/torch/hub/facebookresearch_dinov2_main
/raid/home/detectionfeuxdeforet/caillaud_gab/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/raid/home/detectionfeuxdeforet/caillaud_gab/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/raid/home/detectionfeuxdeforet/caillaud_gab/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [18]:
BATCH_SIZE=64

In [19]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=BATCH_SIZE)

In [20]:
train_dataset = PrecomputedDataset(*precompute(train_dataloader, feature_extractor, device))
val_dataset = PrecomputedDataset(*precompute(val_dataloader, feature_extractor, device))

  0%|          | 0/1563 [00:00<?, ?it/s]

/raid/home/detectionfeuxdeforet/caillaud_gab/dlmi/DLMI_histopathology/dataset.py:34: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  img = torch.tensor(hdf.get(img_id).get('img'))


  0%|          | 0/546 [00:00<?, ?it/s]

In [22]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=BATCH_SIZE)

In [21]:
from DLMI_histopathology.models import HistoClassifierHead

In [23]:
model = HistoClassifierHead(dim_input=384, hidden_dim=64, dropout=0.2)
model.to(device)

HistoClassifierHead(
  (layer1): Linear(in_features=384, out_features=64, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=64, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=1, bias=True)
  (layernorm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  (layernorm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (layernorm3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [24]:
OPTIMIZER = 'AdamW'
OPTIMIZER_PARAMS = {'lr': 5e-4, 'weight_decay' : 0.02}
LOSS = 'BCELoss'
METRIC = 'Accuracy'
NUM_EPOCHS = 100
PATIENCE = 15
optimizer = getattr(torch.optim, OPTIMIZER)(model.parameters(), **OPTIMIZER_PARAMS)
criterion = getattr(torch.nn, LOSS)()
metric = getattr(torchmetrics, METRIC)('binary')
min_loss, best_epoch = float('inf'), 0

In [25]:
train_model(model, train_dataloader, val_dataloader, device, optimizer_name=OPTIMIZER, optimizer_params=OPTIMIZER_PARAMS, 
                 loss_name=LOSS, metric_name=METRIC, num_epochs=NUM_EPOCHS, patience=PATIENCE, save_path='best_model.pth')

Epoch [1/100] Train Loss: 0.1673 | Train Metric 0.9357 Val Loss: 0.4108 Val Metric 0.8546
New best loss inf -> 0.4108


Epoch [2/100] Train Loss: 0.1290 | Train Metric 0.9508 Val Loss: 0.3651 Val Metric 0.8651
New best loss 0.4108 -> 0.3651


Epoch [3/100] Train Loss: 0.1175 | Train Metric 0.9560 Val Loss: 0.4765 Val Metric 0.8506


Epoch [4/100] Train Loss: 0.1095 | Train Metric 0.9584 Val Loss: 0.4357 Val Metric 0.8637


Epoch [5/100] Train Loss: 0.1056 | Train Metric 0.9609 Val Loss: 0.5688 Val Metric 0.8313


...

Epoch [13/100] Train Loss: 0.0852 | Train Metric 0.9676 Val Loss: 0.5305 Val Metric 0.8582


Epoch [14/100] Train Loss: 0.0832 | Train Metric 0.9691 Val Loss: 0.4406 Val Metric 0.8631


Epoch [15/100] Train Loss: 0.0837 | Train Metric 0.9691 Val Loss: 0.4749 Val Metric 0.8567


Epoch [16/100] Train Loss: 0.0811 | Train Metric 0.9692 Val Loss: 0.3965 Val Metric 0.8715


Epoch [17/100] Train Loss: 0.0807 | Train Metric 0.9700 Val Loss: 0.4994 Val Metric 0.8609
Model has not improved in val set for 15 epochs. Stopping early.


In [37]:
model.load_state_dict(torch.load('best_model.pth', weights_only=True))
model.eval()
model.to(device)
prediction_dict = {}

In [38]:
with h5py.File(TEST_IMAGES_PATH, 'r') as hdf:
    test_ids = list(hdf.keys())

In [39]:
solutions_data = {'ID': [], 'Pred': []}
with h5py.File(TEST_IMAGES_PATH, 'r') as hdf:
    for test_id in tqdm(test_ids):
        img = np.array(hdf.get(test_id).get('img'))
        img = transform(torch.tensor(img)).unsqueeze(0).float()
        pred = model(feature_extractor(img.to(device))).detach().cpu()
        solutions_data['ID'].append(int(test_id))
        solutions_data['Pred'].append(int(pred.item() > 0.5))
solutions_data = pd.DataFrame(solutions_data).set_index('ID')
solutions_data.to_csv('baseline.csv')

  0%|          | 0/85054 [00:00<?, ?it/s]

## test en prenant le modèle qui performe le mieux sur la val metric plutôt que sur la val loss

In [36]:
model = HistoClassifierHead(dim_input=384, hidden_dim=64, dropout=0.2)
model.to(device)
save_path  ="best_model.pth"
optimizer = getattr(torch.optim, "AdamW")(model.parameters(), **OPTIMIZER_PARAMS)
criterion = getattr(torch.nn, "BCELoss")()
metric = getattr(torchmetrics, METRIC)('binary')
num_epochs=NUM_EPOCHS
patience=PATIENCE
best_metric, best_epoch = 0, 0
model.to(device)

for epoch in range(num_epochs):
    model.train()
    train_metrics, train_losses = [], []
    
    for train_x, train_y in tqdm(train_dataloader, leave=False):
        optimizer.zero_grad()
        train_pred = model(train_x.to(device))
        loss = criterion(train_pred, train_y.to(device))
        loss.backward()
        optimizer.step()
        
        train_losses.extend([loss.item()] * len(train_y))
        train_metric = metric(train_pred.cpu(), train_y.int().cpu())
        train_metrics.extend([train_metric.item()] * len(train_y))
    
    model.eval()
    val_metrics, val_losses = [], []
    
    for val_x, val_y in tqdm(val_dataloader, leave=False):
        with torch.no_grad():
            val_pred = model(val_x.to(device))
        loss = criterion(val_pred, val_y.to(device))
        
        val_losses.extend([loss.item()] * len(val_y))
        val_metric = metric(val_pred.cpu(), val_y.int().cpu())
        val_metrics.extend([val_metric.item()] * len(val_y))
    
    print(f'Epoch [{epoch+1}/{num_epochs}] Train Loss: {np.mean(train_losses):.4f} | Train Metric {np.mean(train_metrics):.4f} '
            f'Val Loss: {np.mean(val_losses):.4f} Val Metric {np.mean(val_metrics):.4f}')
    
    if np.mean(val_metrics)> best_metric:
        mean_val_metric = np.mean(val_metrics)
        print(f'New best metric  {best_metric:.4f} -> {mean_val_metric:.4f}')
        best_metric = mean_val_metric
        best_epoch = epoch
        torch.save(model.state_dict(), save_path)
    
    if epoch - best_epoch == patience:
        print(f"Model has not improved in val set for {patience} epochs. Stopping early.")
        break

  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/546 [00:00<?, ?it/s]

Epoch [1/100] Train Loss: 0.1688 | Train Metric 0.9359 Val Loss: 0.4279 Val Metric 0.8600
New best metric  0.0000 -> 0.8600


  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/546 [00:00<?, ?it/s]

...

  0%|          | 0/546 [00:00<?, ?it/s]

Epoch [33/100] Train Loss: 0.0707 | Train Metric 0.9740 Val Loss: 0.5210 Val Metric 0.8508


  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/546 [00:00<?, ?it/s]

Epoch [34/100] Train Loss: 0.0703 | Train Metric 0.9744 Val Loss: 0.4656 Val Metric 0.8641
Model has not improved in val set for 15 epochs. Stopping early.


Testing Macenko's stain normalization

In [ ]:
import staintools
import cv2

# Load the target image (reference image)
target_image = cv2.imread("target_image.jpg")

# Load the image to be normalized
source_image = cv2.imread("source_image.jpg")

# Convert images to RGB (OpenCV loads in BGR)
target_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB)
source_image = cv2.cvtColor(source_image, cv2.COLOR_BGR2RGB)

# Instantiate a stain normalizer
normalizer = staintools.StainNormalizer(method='macenko')

# Fit normalizer to target (reference) image
normalizer.fit(target_image)

# Normalize the source image
normalized_image = normalizer.transform(source_image)

In [40]:
source_image = np.load("image_center_0_healthy.npy")   # (H, W, 3), dtype=np.uint8